In [1]:
from typedspark import Schema, Column, StructType
from pyspark.sql.types import IntegerType, StringType

class A(Schema):
    alpha: Column[IntegerType]
    bravo: Column[StringType]

class B(Schema):
    a: Column[StructType[A]]
    b: Column[StringType]

RecursionError: maximum recursion depth exceeded in comparison